In [5]:
import cv2
import os
import json

# パラメータファイルの読み込み
with open("parameters.json", "r") as f:
    params = json.load(f)

# 入力動画フォルダと出力画像フォルダの設定
input_folder = params["input_folder"]
output_folder = params["output_folder"]

# 出力フォルダが存在しない場合は作成
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# 動画ファイルの処理
for video_file in os.listdir(input_folder):
    if video_file.endswith((".mp4", ".avi", ".mov")):  # 対応する動画形式を追加
        video_path = os.path.join(input_folder, video_file)
        cap = cv2.VideoCapture(video_path)

        frame_count = 0
        while True:
            ret, frame = cap.read()
            if not ret:
                break

            if frame_count % params["frame_interval"] == 0:
                output_path = os.path.join(
                    output_folder, f"{video_file[:-4]}_{frame_count:06d}.jpg"
                )
                cv2.imwrite(output_path, frame)

            frame_count += 1

        cap.release()

print("処理が完了しました。")

処理が完了しました。


In [94]:
from pptx import Presentation
from pptx.util import Pt, Cm
from pptx.enum.text import PP_ALIGN
from pptx.enum.shapes import MSO_SHAPE
from pptx.dml.color import RGBColor
import os
import json
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
import datetime

# 定数の定義
SLIDE_WIDTH = Cm(21)
SLIDE_HEIGHT = Cm(29.7)
TABLE_HEIGHT = Cm(11)  # 表の高さを短くする


def load_parameters(file_path):
    with open(file_path, "r") as f:
        return json.load(f)


def create_colorbar(min_value, max_value):
    fig, ax = plt.subplots(figsize=(0.5, 5))  # 幅を0.5に変更してカラーバーを細くする
    gradient = np.linspace(min_value, max_value, 256).reshape(256, 1)
    ax.imshow(gradient, aspect="auto", cmap="jet_r", extent=[0, 1, min_value, max_value])
    ax.yaxis.set_label_position("right")
    ax.xaxis.set_visible(False)
    ax.yaxis.tick_right()
    
    # フォントサイズを大きくする
    ax.tick_params(axis='y', labelsize=20)  # ここでフォントサイズを調整
    
    # 上限値と下限値を必ず表示する
    ax.set_yticks([min_value, max_value])
    ax.set_yticklabels([f"{min_value:.1f}", f"{max_value:.1f}"])
    
    plt.tight_layout()

    temp_file = "temp_colorbar.png"
    plt.savefig(temp_file, dpi=300, bbox_inches="tight")
    plt.close()

    return temp_file


def group_images_by_video(image_folder, include_first_frame):
    image_files = [
        f for f in os.listdir(image_folder) if f.endswith((".jpg", ".png", ".jpeg"))
    ]
    image_groups = {}
    for image_file in image_files:
        video_name = image_file.split("_")[0]
        frame_number = int(image_file.split("_")[-1].split(".")[0])
        if not include_first_frame and frame_number == 0:
            continue
        if video_name not in image_groups:
            image_groups[video_name] = []
        image_groups[video_name].append(image_file)
    return image_groups


def add_grid_lines(
    slide, table_left, table_width, title_top, cell_width, cell_height, rows, cols
):
    for row in range(rows + 1):
        line = slide.shapes.add_shape(
            MSO_SHAPE.RECTANGLE,
            table_left,
            title_top + row * cell_height,
            table_width,
            0,
        )
        line.fill.background()
        line.line.color.rgb = RGBColor(0, 0, 0)

    for col in range(cols + 1):
        line = slide.shapes.add_shape(
            MSO_SHAPE.RECTANGLE,
            table_left + col * cell_width,
            title_top,
            0,
            rows * cell_height,
        )
        line.fill.background()
        line.line.color.rgb = RGBColor(0, 0, 0)


def calculate_table_height(cell_width, rows, image_aspect_ratio):
    cell_height = cell_width / image_aspect_ratio
    frame_text_height = Cm(0.7)  # フレーム数の文字の高さ
    return (cell_height + frame_text_height) * rows


def format_time(seconds):
    return str(datetime.timedelta(seconds=seconds)).split('.')[0]

def add_image_to_slide(slide, img_path, left, top, cell_width, cell_height, seconds_per_frame):
    with Image.open(img_path) as img:
        img_width, img_height = img.size

    aspect_ratio = img_width / img_height
    image_height = cell_height - Cm(0.7)  # フレーム数の文字の高さを引く
    image_width = image_height * aspect_ratio

    if image_width > cell_width:
        image_width = cell_width
        image_height = image_width / aspect_ratio

    image_left = left + (cell_width - image_width) / 2
    image_top = top

    slide.shapes.add_picture(
        img_path, image_left, image_top, width=image_width, height=image_height
    )

    frame_number = int(os.path.basename(img_path).split("_")[-1].split(".")[0])
    elapsed_time = format_time(frame_number * seconds_per_frame)
    txBox = slide.shapes.add_textbox(
        left, top + image_height, cell_width, Cm(0.7)
    )
    tf = txBox.text_frame
    tf.text = f"{elapsed_time.split(':')[0]}:{elapsed_time.split(':')[1]}, frame:{frame_number}"
    tf.paragraphs[0].alignment = PP_ALIGN.CENTER
    tf.paragraphs[0].font.size = Pt(10)  # フォントサイズを小さくして2行に収める


def create_presentation(params_list):
    prs = Presentation()
    prs.slide_width = SLIDE_WIDTH
    prs.slide_height = SLIDE_HEIGHT
    blank_slide_layout = prs.slide_layouts[6]

    # グローバルパラメータの設定（最初の要素をデフォルトとして使用）
    global_params = params_list[0] if params_list else {}
    
    # デフォルト値の設定
    default_params = {
        "show_colorbar": True,
        "images_per_video": 8,
        "rows": 2,
        "cols": 4,
        "include_first_frame": True,
        "seconds_per_frame": 360,
        "min_threshold": 0,
        "max_threshold": 255,
        "output_folder": ""
    }

    # グローバルパラメータにデフォルト値を適用
    for key, value in default_params.items():
        if key not in global_params:
            global_params[key] = value

    image_groups = group_images_by_video(global_params["output_folder"], global_params["include_first_frame"])
    video_names = list(image_groups.keys())

    for i, params in enumerate(params_list):
        if i % 2 == 0:
            slide = prs.slides.add_slide(blank_slide_layout)
        
        video_index = i % 2

        # 現在のパラメータにグローバルパラメータを適用し、その後個別のパラメータで上書き
        current_params = global_params.copy()
        current_params.update(params)

        table_left = Cm(0.5)
        available_height = SLIDE_HEIGHT / 2 - Cm(2)

        if current_params["show_colorbar"]:
            colorbar_file = create_colorbar(
                current_params["min_threshold"],
                current_params["max_threshold"]
            )
            colorbar_left, colorbar_top = Cm(0.5), video_index * (SLIDE_HEIGHT / 2) + Cm(1)
            colorbar_width, colorbar_height = Cm(1.5), available_height / 2  # 高さを半分に変更
            slide.shapes.add_picture(
                colorbar_file,
                colorbar_left,
                colorbar_top,
                width=colorbar_width,
                height=colorbar_height,
            )
            os.remove(colorbar_file)
            table_left = colorbar_left + colorbar_width + Cm(0.5)


        table_width = SLIDE_WIDTH - table_left - Cm(0.5)
        cell_width = table_width / current_params["cols"]

        # 画像のアスペクト比を取得（最初の画像を使用）
        if i < len(video_names):
            video_name = video_names[i]
            first_image = os.path.join(current_params["output_folder"], image_groups[video_name][0])
            with Image.open(first_image) as img:
                image_aspect_ratio = img.width / img.height
        else:
            # ビデオが存在しない場合のデフォルト値
            image_aspect_ratio = 16/9  # 一般的な動画のアスペクト比

        table_height = calculate_table_height(cell_width, current_params["rows"], image_aspect_ratio)
        cell_height = table_height / current_params["rows"]

        title_top = video_index * (SLIDE_HEIGHT / 2)
        title = slide.shapes.add_textbox(Cm(1), title_top, SLIDE_WIDTH - Cm(2), Cm(1))
        title.text = f"Video {i+1}" if i < len(video_names) else f"Video {i+1} (No data)"
        title.text_frame.paragraphs[0].font.size = Pt(18)
        title.text_frame.paragraphs[0].font.bold = True

        add_grid_lines(
            slide,
            table_left,
            table_width,
            title_top + Cm(1),
            cell_width,
            cell_height,
            current_params["rows"],
            current_params["cols"]
        )

        if i < len(video_names):
            group_images = sorted(
                image_groups[video_name],
                key=lambda x: int(x.split("_")[-1].split(".")[0]),
            )
            for j, img_file in enumerate(group_images[:current_params["images_per_video"]]):
                img_path = os.path.join(current_params["output_folder"], img_file)
                row, col = j // current_params["cols"], j % current_params["cols"]
                left = table_left + col * cell_width
                top = title_top + Cm(1) + row * cell_height
                add_image_to_slide(slide, img_path, left, top, cell_width, cell_height, current_params["seconds_per_frame"])

    prs.save("image_summary_a4_two_videos_improved.pptx")

if __name__ == "__main__":
    params_list = load_parameters("parameters_list.json")
    create_presentation(params_list)
    print("プレゼンテーションが作成されました。")

/tmp/ipykernel_1288/519050087.py:40: UserWarning: Tight layout not applied. The left and right margins cannot be made large enough to accommodate all axes decorations.
  plt.tight_layout()
/tmp/ipykernel_1288/519050087.py:40: UserWarning: Tight layout not applied. The left and right margins cannot be made large enough to accommodate all axes decorations.
  plt.tight_layout()


プレゼンテーションが作成されました。


In [16]:
# from pptx import Presentation
# from pptx.util import Inches, Pt, Cm
# from pptx.enum.text import PP_ALIGN
# from pptx.enum.shapes import MSO_SHAPE
# from pptx.dml.color import RGBColor
# import os
# import json
# from PIL import Image
# import numpy as np
# import matplotlib.pyplot as plt
# from matplotlib import cm

# # パラメータファイルの読み込み
# with open("parameters.json", "r") as f:
#     params = json.load(f)

#     # スケールバーの作成関数


# def create_colorbar(min_value, max_value, height):
#     fig, ax = plt.subplots(figsize=(1, 10))
#     gradient = np.linspace(min_value, max_value, 256).reshape(256, 1)
#     ax.imshow(gradient, aspect="auto", cmap="jet", extent=[0, 1, min_value, max_value])
#     # ax.set_ylabel('統計量')
#     ax.yaxis.set_label_position("right")
#     ax.yaxis.tick_right()
#     plt.tight_layout()

#     # 一時ファイルに保存
#     temp_file = "temp_colorbar.png"
#     plt.savefig(temp_file, dpi=300, bbox_inches="tight")
#     plt.close()

#     return temp_file


# # 閾値の取得
# min_threshold = params.get("min_threshold", 0)
# max_threshold = params.get("max_threshold", 255)

# # 画像フォルダの設定
# image_folder = params["output_folder"]

# # 新しいプレゼンテーションの作成（A4縦サイズ）
# prs = Presentation()
# prs.slide_width = Cm(21)
# prs.slide_height = Cm(29.7)

# # スライドのレイアウトを選択（空白のレイアウト）
# blank_slide_layout = prs.slide_layouts[6]

# # 画像ファイルのリストを取得し、動画ごとにグループ化
# image_files = [
#     f for f in os.listdir(image_folder) if f.endswith((".jpg", ".png", ".jpeg"))
# ]
# image_groups = {}
# for image_file in image_files:
#     video_name = image_file.split("_")[0]
#     if video_name not in image_groups:
#         image_groups[video_name] = []
#     image_groups[video_name].append(image_file)

# # 画像を4x2のグリッドで配置（1動画につき）
# images_per_video = 8
# rows, cols = 2, 4

# # 動画のリストを2つずつ処理
# video_names = list(image_groups.keys())
# for i in range(0, len(video_names), 2):
#     # 新しいスライドを追加
#     slide = prs.slides.add_slide(blank_slide_layout)

#     for video_index, video_name in enumerate(video_names[i : i + 2]):
#         if video_name not in image_groups:
#             continue
#         # 各動画用のカラーバーを作成
#         colorbar_file = create_colorbar(
#             min_threshold, max_threshold, prs.slide_height / 2 - Cm(2)
#         )
#         colorbar_left = Cm(0.5)
#         colorbar_top = video_index * (prs.slide_height / 2) + Cm(1.5)
#         colorbar_width = Cm(1.5)
#         colorbar_height = prs.slide_height / 2 - Cm(2)
#         slide.shapes.add_picture(
#             colorbar_file,
#             colorbar_left,
#             colorbar_top,
#             width=colorbar_width,
#             height=colorbar_height,
#         )
#         os.remove(colorbar_file)

#         # 既存の表の位置を調整
#         table_left = colorbar_left + colorbar_width + Cm(0.5)
#         table_width = prs.slide_width - table_left - Cm(0.5)

#         group_images = sorted(
#             image_groups[video_name], key=lambda x: int(x.split("_")[-1].split(".")[0])
#         )

#         # タイトルを追加
#         title_top = video_index * (prs.slide_height / 2)
#         title = slide.shapes.add_textbox(
#             Cm(1), title_top, prs.slide_width - Cm(2), Cm(1.5)
#         )
#         title.text = f"動画: {video_name}"
#         title.text_frame.paragraphs[0].font.size = Pt(18)  # タイトルの文字を大きく
#         title.text_frame.paragraphs[0].font.bold = True

#         # セルのサイズを計算（調整後）
#         cell_width = table_width / cols
#         cell_height = (prs.slide_height / 2 - Cm(2)) / rows  # 縦の長さを少し短く

#         # 表の枠線を追加（位置を調整）
#         for row in range(rows + 1):
#             line = slide.shapes.add_shape(
#                 MSO_SHAPE.RECTANGLE,
#                 table_left,
#                 title_top + Cm(1.5) + row * cell_height,
#                 table_width,
#                 0,
#             )
#             line.fill.background()
#             line.line.color.rgb = RGBColor(0, 0, 0)

#         for col in range(cols + 1):
#             line = slide.shapes.add_shape(
#                 MSO_SHAPE.RECTANGLE,
#                 table_left + col * cell_width,
#                 title_top + Cm(1.5),
#                 0,
#                 rows * cell_height,
#             )
#             line.fill.background()
#             line.line.color.rgb = RGBColor(0, 0, 0)

#         for j, img_file in enumerate(group_images[:images_per_video]):
#             img_path = os.path.join(image_folder, img_file)
#             row = j // cols
#             col = j % cols

#             left = table_left + col * cell_width
#             top = title_top + Cm(1.5) + row * cell_height

#             # 画像のサイズを取得
#             with Image.open(img_path) as img:
#                 img_width, img_height = img.size

#             # アスペクト比を維持しながら、セルいっぱいに画像を配置
#             aspect_ratio = img_width / img_height
#             if aspect_ratio > cell_width / cell_height:
#                 width = cell_width
#                 height = width / aspect_ratio
#             else:
#                 height = cell_height
#                 width = height * aspect_ratio

#             # 画像をセンタリング
#             left += (cell_width - width) / 2
#             top += (cell_height - height) / 2

#             # 画像を追加
#             slide.shapes.add_picture(img_path, left, top, width=width, height=height)

#             # フレーム番号を追加
#             frame_number = int(img_file.split("_")[-1].split(".")[0])
#             txBox = slide.shapes.add_textbox(
#                 table_left + col * cell_width, top + height, cell_width, Cm(0.5)
#             )
#             tf = txBox.text_frame
#             tf.text = f"フレーム: {frame_number}"
#             tf.paragraphs[0].alignment = PP_ALIGN.CENTER
#             tf.paragraphs[0].font.size = Pt(8)

# # プレゼンテーションを保存
# prs.save("image_summary_a4_two_videos_improved.pptx")

# print(
#     "A4縦サイズで、2つの動画を1スライドに配置し、アスペクト比を維持し、フレーム番号と動画タイトルを追加し、表の枠線を追加したパワーポイントのまとめが作成されました。"
# )

A4縦サイズで、2つの動画を1スライドに配置し、アスペクト比を維持し、フレーム番号と動画タイトルを追加し、表の枠線を追加したパワーポイントのまとめが作成されました。
